<a href="https://colab.research.google.com/github/a-apte/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

#### Loading Stage

In [0]:
import numpy as np
import pandas as pd
from scipy import stats


In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data'

col_names = ['A1','A2','A3','A4','A5','A6','A7','A8', 'A9',
             'A10','A11','A12','A13','A14','A15','A16']

df_raw = pd.read_csv(url, header = None, names = col_names)

print (df_raw.shape)

df_raw.head()


(690, 16)


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


#### Validation Stage

7.  Attribute Information:

    A1:	b, a.
    
    A2:	continuous.
    
    A3:	continuous.
    
    A4:	u, y, l, t.
    
    A5:	g, p, gg.
    
    A6:	c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
    
    A7:	v, h, bb, j, n, z, dd, ff, o.
    
    A8:	continuous.
    
    A9:	t, f.
    
    A10:	t, f.
    
    A11:	continuous.
    
    A12:	t, f.
    
    A13:	g, p, s.
    
    A14:	continuous.
    
    A15:	continuous.
    
    A16: +,-         (class attribute)

8.  Missing Attribute Values:

    37 cases (5%) have one or more missing values.  The missing values from particular attributes are:

    A1:  12
    
    A2:  12
    
    A4:   6
    
    A5:   6
    
    A6:   9
    
    A7:   9
    
    A14: 13

9.  Class Distribution
  
    +: 307 (44.5%)
    
    -: 383 (55.5%)

In [3]:
df_raw.isnull().sum()

A1     0
A2     0
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
A16    0
dtype: int64

#### Converting the categorical variables into something friendlier

In [4]:
df_raw.A1.unique()

array(['b', 'a', '?'], dtype=object)

There are no null values because the nulls have been replaced by '?', the next step will fix that.

In [5]:
df_raw.replace('?', np.nan, inplace = True)
df_raw.isnull().sum()

#Bingo

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

In [0]:
# replace_values = {'-' : 0, '+' : 1}

# df_raw.A16.replace(replace_values, inplace = True)

# df_raw.A16.unique()

In [7]:
replace_values_tf = {'t' : True, 'f' : False}

df_raw.A9.replace(replace_values_tf, inplace = True)
df_raw.A10.replace(replace_values_tf, inplace = True)
df_raw.A12.replace(replace_values_tf, inplace = True)

print(df_raw.A9.unique())
print(df_raw.A10.unique())
print(df_raw.A12.unique())

[ True False]
[ True False]
[False  True]


#### Fixing the continuous variables

In [8]:
df_raw.describe() #Missing continuous variables

,A3,A8,A11,A15
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [9]:
df_raw.dtypes # Two continuous variables need to be converted

A1      object
A2      object
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9        bool
A10       bool
A11      int64
A12       bool
A13     object
A14     object
A15      int64
A16     object
dtype: object

In [10]:
df_raw.A2 = df_raw.A2.astype(float)
df_raw.A14 = df_raw.A14.astype(float)

df_raw.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9        bool
A10       bool
A11      int64
A12       bool
A13     object
A14    float64
A15      int64
A16     object
dtype: object

In [11]:
df_raw.describe() # Now 6 continuous variables

,A2,A3,A8,A11,A14,A15
count,678.000000,690.000000,690.000000,690.00000,677.000000,690.000000
mean,31.568171,4.758725,2.223406,2.40000,184.014771,1017.385507
std,11.957862,4.978163,3.346513,4.86294,173.806768,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,22.602500,1.000000,0.165000,0.00000,75.000000,0.000000
50%,28.460000,2.750000,1.000000,0.00000,160.000000,5.000000
75%,38.230000,7.207500,2.625000,3.00000,276.000000,395.500000
max,80.250000,28.000000,28.500000,67.00000,2000.000000,100000.000000


In [12]:
df_raw.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,True,True,1,False,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,True,True,6,False,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,True,False,0,False,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,True,True,5,True,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,True,False,0,False,s,120.0,0,+


#### Handling the Null values

In this case, since the loss of observations is small and the dataset is large enough to perform statistics, the simplest way to deal with the Nan values is to drop all of them.

In [13]:
df_raw.dropna().shape



(653, 16)

In [14]:
df_raw.shape

(690, 16)

In [15]:
df = df_raw.dropna()
df.isnull().sum()

A1     0
A2     0
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
A16    0
dtype: int64

In [16]:
df.shape

(653, 16)

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


# Note: only works with printing strings, print(color.BOLD + str + color.END)

#### T-tests of different means

**H0: the differences in means of variables between the plus and minus group can be explained by chance.**

**HA: the differences in means of variables between the plus and minus group cannot be explained by chance.**

**One rejects the null if p < 0.025 (Two-tailed test)**

In [51]:
plus_df = df[df.A16 == '+']
minus_df = df[df.A16 == '-']

plus = len(plus_df)
minus = len(minus_df)

print (plus)
print (minus)

print ((plus)/(plus+minus)) # Original 44.5%
print ((minus)/(plus+minus)) # Original 55.5%

296
357
0.45329249617151607
0.5467075038284839


In [88]:
results = []

cols =  ['A2', 'A3', 'A8', 'A11', 'A14', 'A15']

for col in cols: 
#   plus_col = np.array(plus_df[col])
#   minus_col = np.array(minus_df[col])
  plus_mean = plus_df[col].mean()
  minus_mean = minus_df[col].mean()
  t_stat = stats.ttest_ind(plus_df[col], minus_df[col],equal_var = False, axis = None).statistic
  p_val = stats.ttest_ind(plus_df[col], minus_df[col],equal_var = False, axis = None).pvalue
#   print (col)
  results.append({'0_Column':col, '1_+_mean': plus_mean, '2_-_mean': minus_mean, '3_T_Statistic': t_stat, '4_P_Value': p_val})

pd.DataFrame(results)

,0_Column,1_+_mean,2_-_mean,3_T_Statistic,4_P_Value
0,A2,33.845473,29.562269,4.603106,5.118622e-06
1,A3,5.971943,3.882325,5.290896,1.748909e-07
2,A8,3.475186,1.223725,8.497112,3.654047e-16
3,A11,4.716216,0.666667,10.489489,1.702907e-22
4,A14,164.621622,193.408964,-2.193852,2.860489e-02
5,A15,2009.726351,187.974790,4.079694,5.792817e-05


#### Cross-Tabulations with the Class Variable (+/-) and the categorical variables and Chi-Squared tests

**Chi-squared test**

**H0: the rows and columns of the cross table are _independent_ and therefore the results are explained by chance.**

**HA: the rows and columns of the cross table are _dependent_ and therefore the results cannot be explained by chance**

**Once rejects the null hypothesis if p < 0.05 (one-tailed test)**

In [46]:
cross1 = pd.crosstab(df.A16, df.A1)

print(stats.chisquare(cross1, axis=None), "\n")

cross1

Power_divergenceResult(statistic=101.00306278713629, pvalue=9.458202945413513e-22) 



A1,a,b
A16,,
+,95,201
-,108,249


In [36]:
cross4 = pd.crosstab(df.A16, df.A4)

print(stats.chisquare(cross4, axis=None), "\n")

cross4

Power_divergenceResult(statistic=614.800918836141, pvalue=1.2814886836851252e-130) 



A4,l,u,y
A16,,,
+,2,249,45
-,0,250,107


In [37]:
cross5 = pd.crosstab(df.A16, df.A5)

print(stats.chisquare(cross5, axis=None), "\n")

cross5 

Power_divergenceResult(statistic=614.800918836141, pvalue=1.2814886836851252e-130) 



A5,g,gg,p
A16,,,
+,249,2,45
-,250,0,107


In [38]:
cross6 = pd.crosstab(df.A16, df.A6)

print(stats.chisquare(cross6, axis=None), "\n")

cross6

Power_divergenceResult(statistic=376.99693721286366, pvalue=2.365893078307223e-63) 



A6,aa,c,cc,d,e,ff,i,j,k,m,q,r,w,x
A16,,,,,,,,,,,,,,
+,19,60,29,7,14,7,14,3,13,16,49,2,33,30
-,33,73,11,19,10,43,41,7,35,22,26,1,30,6


In [39]:
cross7 = pd.crosstab(df.A16, df.A7)

print(stats.chisquare(cross7, axis=None), "\n")

cross7

Power_divergenceResult(statistic=1768.9509954058192, pvalue=0.0) 



A7,bb,dd,ff,h,j,n,o,v,z
A16,,,,,,,,,
+,24,2,8,87,3,2,1,163,6
-,29,4,46,50,5,2,1,218,2


In [40]:
cross9 = pd.crosstab(df.A16, df.A9)

print(stats.chisquare(cross9, axis=None), "\n")

cross9

Power_divergenceResult(statistic=354.32006125574276, pvalue=1.730751761550774e-76) 



A9,False,True
A16,,
+,18,278
-,286,71


In [41]:
cross10 = pd.crosstab(df.A16, df.A10)

print(stats.chisquare(cross10, axis=None), "\n")

cross10

Power_divergenceResult(statistic=152.16385911179174, pvalue=8.994150430808612e-33) 



A10,False,True
A16,,
+,93,203
-,273,84


In [42]:
cross12 = pd.crosstab(df.A16, df.A12)

print(stats.chisquare(cross12, axis=None), "\n")

cross12

Power_divergenceResult(statistic=11.47166921898928, pvalue=0.009430579350519394) 



A12,False,True
A16,,
+,151,145
-,200,157


In [43]:
cross13 = pd.crosstab(df.A16, df.A13)

print(stats.chisquare(cross13, axis=None), "\n")

cross13

Power_divergenceResult(statistic=1011.8851454823891, pvalue=1.605961312555435e-216) 



A13,g,p,s
A16,,,
+,280,1,15
-,318,1,38


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

**T-tests**: For every continuous variable, the difference in means between the plus and minus group is *statistically significant*. Therefore, one can **reject the null hypothesis**. For the plus group, it has greater amounts of A2, A3, A8, A11 and A15 and a smaller amount of A14 than the minus group which are all statistically significant. A11 is the most different and A14 is the least different.

**Chi squared test**: In every case, one can **reject the null hypothesis**. Each set of cross tab results is *statistically significant*.

**Most challenging part**: I didn't find this too challenging. I did have to lookup from previous work the specific stats commands but that wasn't too much of an issue. The only  challenge I believe I faced was the decision on what to do with the null values for which there is no one correct answer. I am comfortable with my treatment of the null values in this dataset.

